# Introduction – Contexte  du projet

Dans un marché immobilier en constante évolution, prendre une décision d’investissement éclairée repose sur une compréhension fine des dynamiques locales : prix au m², attractivité des quartiers, proximité des services, ou encore potentiel locatif.
Grâce aux données foncières publiques (DVF) mises à disposition par le gouvernement français, il est aujourd’hui possible d’analyser ces tendances de manière précise et objective.
Cette étude a pour objectif de fournir une analyse complète du marché immobilier afin d’identifier les zones les plus attractives et rentables pour un investisseur souhaitant se positionner sur le locatif résidentiel.
Elle s’appuie sur une approche data-driven, combinant le traitement, la visualisation et l’interprétation des données immobilières issues des ventes réelles.

## Objectif de l'étude 

L’objectif est de proposer à l’investisseur : 
Une vision claire du marché immobilier dans plusieurs zones urbaines et périurbaines françaises ;
Une évaluation de la rentabilité potentielle en fonction du prix d’achat, du type de bien et du profil locatif visé ;
Des recommandations précises sur les zones à privilégier pour maximiser la rentabilité et minimiser le risque locatif.




# 👩‍💼 Persona – Sara

Sara, 28 ans, souhaite réaliser son premier investissement locatif.  
Active et prévoyante, elle désire placer son épargne dans un projet immobilier **simple, rentable et durable**.  
Son objectif est d’acquérir un bien **sans travaux**, situé à proximité d’une **grande métropole**, afin de bénéficier d’une **forte demande locative** et de limiter les risques de vacance.

Sa **motivation principale** est de réaliser un **investissement concret et sécurisé**, dans une zone où la demande locative est naturellement soutenue.  
Elle privilégie la **stabilité** et la **rentabilité à long terme**, plutôt que la spéculation à court terme.

---

<div style="display:flex; align-items:flex-start; gap:25px; margin-top:15px;">

  <img src="avatar_Sara.jpeg" alt="Avatar Sara" width="170" style="border-radius:50%; box-shadow: 0 0 10px rgba(0,0,0,0.25);">

  <div style="line-height:1.7;">

  ### 👩 Informations personnelles
  - **Âge :** 28 ans  
  - **Situation :** jeune cadre salariée dans le secteur tertiaire  
  - **Localisation actuelle :** Lyon  
  - **Statut :** primo-investisseuse  
  - **Revenu mensuel net :** environ 2 500 €  

  ### 🎯 Objectifs d’investissement
  - Réaliser un **premier placement locatif sécurisé**  
  - Créer une **source de revenus complémentaires**  
  - Se constituer un **patrimoine immobilier** sur le long terme  
  - Trouver un bien **clé en main**, sans travaux ni gestion complexe  

  ### 💰 Budget et contraintes
  - **Budget global :** 100 000 à 150 000 € (frais inclus)  
  - Aucun travaux à prévoir (ni rénovation, ni aménagement lourd)  
  - Investissement à crédit avec un apport modéré (5 à 10 %)  
  - Rendement brut minimum recherché : **≥ 5,5 %**

  ### 🏙️ Localisation et critères de recherche
  Sara cible prioritairement les **zones proches de grandes métropoles** où la demande locative est forte :  
  - Banlieues étudiantes, villes universitaires, quartiers desservis par les transports  
  - Bonne accessibilité, commerces à proximité, environnement sécurisé  

  ### 🏠 Typologie de biens recherchés
  **1. Investissement étudiant :**  
  Studio ou T1 de 20 à 30 m², situé à moins de **20 minutes à pied d’une université ou d’une grande école**, meublé ou prêt à louer.  
  Objectif : **rentabilité élevée**, avec un **turn-over locatif accepté**.  

  **2. Investissement jeune couple :**  
  T2 ou petit T3 de 40 à 55 m², avec **deux chambres**, dans un environnement calme et familial, proche des écoles et services.  
  Objectif : **stabilité locative sur le long terme**.  

  ### 💬 Sa demande
  > “Je veux un bien qui se loue facilement, sans avoir à gérer des travaux ou des imprévus.  
  > L’idée, c’est de construire un patrimoine qui me rapporte dès aujourd’hui.”

  </div>
</div>






### Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
from sklearn.neighbors import BallTree

Notebook configuration

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)  

### Data Collect

Toutes les données viennent de ces liens.

- **Loyers eu m2 pour T1 et T2** : [https://www.data.gouv.fr/datasets/carte-des-loyers-indicateurs-de-loyers-dannonce-par-commune-en-2023/](https://www.data.gouv.fr/datasets/carte-des-loyers-indicateurs-de-loyers-dannonce-par-commune-en-2022/)
- **Arret de transport** : https://www.data.gouv.fr/datasets/arrets-de-transport-en-france/
- **Point culturel** : https://www.data.gouv.fr/datasets/base-des-lieux-et-equipements-culturels-basilic/
- **Etablissements d'enseignement des premier et second degrés** : https://data.education.gouv.fr/explore/dataset/fr-en-adresse-et-geolocalisation-etablissements-premier-et-second-degre/export/?disjunctive.numero_uai&disjunctive.code_departement&disjunctive.code_region&disjunctive.code_academie&disjunctive.nature_uai&disjunctive.nature_uai_libe&disjunctive.code_commune&disjunctive.libelle_departement&disjunctive.libelle_region&disjunctive.libelle_academie&disjunctive.secteur_prive_code_type_contrat&disjunctive.secteur_prive_libelle_type_contrat&disjunctive.code_ministere&disjunctive.libelle_ministere
- **Valeurs foncière** : https://www.data.gouv.fr/datasets/demandes-de-valeurs-foncieres-geolocalisees/
- **Prix et indices de prix à la consommation** (Carburant) : https://www.data.gouv.fr/datasets/prix-des-carburants-en-france-flux-instantane-v2-amelioree/
- détail des loyers et autres par arrondissement de paris
- https://www.data.gouv.fr/datasets/communes-de-france-base-des-codes-postaux

Elles sont récupérables via le lien Google Drive suivant : https://drive.google.com/file/d/1YN-JBqsOTZcvSHNrJZWFY7XpdvYqjkUx/view?usp=sharing

### Preprocessing

In [ ]:
chunks = pd.read_csv('../Data/Carburant.csv', chunksize=1000000, low_memory=False, sep=";")

chunk_list = []
for chunk in chunks:
    print("Processing a new chunk...")
    df1 = pd.DataFrame(chunk)
    df1.columns = df1.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
    chunk_list.append(df1[['code_postal', 'prix_gazole', 'prix_sp95' , 'prix_e85', 'prix_gplc', 'prix_sp98' , 'prix_e10']])

# Concatenate all chunks into a single DataFrame
df = pd.concat(chunk_list, ignore_index=True)
mean = df.groupby("code_postal").mean().reset_index()
mean['totmean'] = mean[['prix_gazole', 'prix_sp95' , 'prix_e85', 'prix_sp98' , 'prix_e10']].where(mean[['prix_gazole', 'prix_sp95' , 'prix_e85', 'prix_sp98' , 'prix_e10']] > 0).mean(axis=1)
mean['code_postal'] = mean['code_postal'].apply(lambda row: str(row).zfill(5))

# Display basic information about the dataset
print(mean.info())
print(mean.sort_values(by="prix_gazole", ascending=False).head(10))
mean.to_csv('../DataCleaned/Carburant_Cleaned.csv', index=False)

In [ ]:
code_postal_ville = [
    #Ile-de-France
    "75000",
    "75001", "75002", "75003", "75004", "75005", "75006", "75007", "75008", "75009", "75010",
    "75011", "75012", "75013", "75014", "75015", "75016", "75017", "75018", "75019", "75020",
    "92100", "92110", "92120", "92130", "92140", "92150", "92160", "92170", "92190",
    "92200", "92210", "92220", "92230", "92240", "92250", "92260", "92270", "92290",
    "92300", "92310", "92320", "92330", "92340", "92350", "92360", "92370", "92380", "92390",   "92400", "92410", "92420", "92430", "92440", "92450", "92460", "92470", "92480", "92490",
    "92500", "92510", "92520", "92530", "92540", "92550", "92560", "92570", "92580", "92590",
    "92600", "92610", "92620", "92630", "92640", "92650", "92660", "92670", "92680", "92690",
    "92700", "92710", "92720", "92730", "92740", "92750", "92760", "92770", "92780", "92790",
    "92800", "92810", "92820", "92830", "92840", "92850", "92860", "92870", "92880", "92890",
    "92900", "92910", "92920", "92930", "92940", "92950", "92960", "92970", "92980", "92990",
    "93000", "93100", "93200", "93300", "93400", "93500", "93600", "93700", "93800", "93900",
    "94000", "94100", "94200", "94300", "94400", "94500", "94600", "94700", "94800", "94900",
    
    #Marseille
    "13000", "13001", "13002", "13003", "13004", "13005", "13006", "13007", "13008", "13009",
    "13010", "13011", "13012", "13013",
    
    #Aix-en-Provence
    "13080", "13100", "13190", "13290", "13540", "13590", "13700", "13800", "13990",
    
    #Lyon
    "69000", "69001", "69002", "69003", "69004", "69005", "69006", "69007", "69008", "69009",
    "69100", "69200", "69300", "69400", "69500", "69600", "69700", "69800", "69900",
    
    #Lille
    "59000", "59100", "59200", "59300", "59400", "59500", "59600", "59700", "59800", "59900",
    
    #Bordeaux
    "33000", "33100", "33200", "33300", "33400", "33500", "33600", "33700", "33800", "33900",
    
    #Toulouse
    "31000", "31100", "31200", "31300", "31400", "31500", "31600", "31700", "31800", "31900",
    
    #Nice
    "06000", "06100", "06200", "06300", "06400", "06500", "06600", "06700", "06800", "06900",
    
    #Nantes
    "44000", "44100", "44200", "44300", "44400", "44500", "44600", "44700", "44800", "44900",
    
    #Strasbourg
    "67000", "67100", "67200", "67300", "67400", "67500", "67600", "67700", "67800", "67900",
    
    #Montpellier
    "34000", "34100", "34200", "34300", "34400", "34500", "34600", "34700", "34800", "34900",
    
    #Rennes
    "35000", "35100", "35200", "35300", "35400", "35500", "35600", "35700", "35800", "35900",
    
    #Grenoble
    "38000", "38100", "38200", "38300", "38400", "38500", "38600", "38700", "38800", "38900",
    
    #Dijon
    "21000", "21100", "21200", "21300", "21400", "21500", "21600", "21700", "21800", "21900",   
    
    #Angers
    "49000", "49100", "49200", "49300", "49400", "49500", "49600", "49700", "49800", "49900",
    
    #Rennes
    "35000", "35100", "35200", "35300", "35400", "35500", "35600", "35700", "35800", "35900",
    
    #Le Havre
    "76000", "76100", "76200", "76300", "76400", "76500", "76600", "76700", "76800", "76900",
    
    #Saint-Étienne
    "42000", "42100", "42200", "42300", "42400", "42500", "42600", "42700", "42800", "42900",
]

# === 1) Chemins des fichiers ===
communes_path = "../Data/communes-france-2025.csv"
adj_path = "../Data/communes_adjacentes_2022_toutes.csv"

# === 2) Charger les fichiers (en texte) ===
communes = pd.read_csv(communes_path, dtype=str)
adj = pd.read_csv(adj_path, dtype=str)

# === 3) Vérifier que les colonnes utiles existent ===
# communes: code INSEE, nom de la commune, codes postaux
if not all(c in communes.columns for c in ["code_insee", "nom_standard", "codes_postaux"]):
    raise ValueError("Le fichier communes doit contenir: code_insee, nom_standard, codes_postaux")

# adjacences: code INSEE source, voisins INSEE
if not all(c in adj.columns for c in ["insee", "insee_voisins"]):
    raise ValueError("Le fichier adjacences doit contenir: insee, insee_voisins")

# === 4) Petite fonction pour découper les listes (séparateur '|') ===
def split_pipe(value):
    """Retourne une liste en séparant par '|' (ou liste vide si NaN)."""
    if pd.isna(value):
        return []
    text = str(value).strip()
    if text == "":
        return []
    return [x.strip() for x in text.split("|") if x.strip()]

# === 5) Préparer la table 'communes' ===
# On garde seulement les colonnes utiles, et on "explose" les codes postaux
communes_simple = communes[["code_insee", "nom_standard", "codes_postaux"]].copy()
communes_simple["liste_cp"] = communes_simple["codes_postaux"].apply(split_pipe)
communes_cp = communes_simple.explode("liste_cp", ignore_index=True)  # une ligne par code postal
communes_cp = communes_cp.rename(columns={
    "nom_standard": "commune",
    "liste_cp": "code_postal"
})

# === 6) Préparer la table 'adjacences' ===
# On "explose" les voisins INSEE (une ligne par voisin)
adj["voisin_insee"] = adj["insee_voisins"].apply(split_pipe)
adj_long = adj.explode("voisin_insee", ignore_index=True)
adj_long = adj_long.dropna(subset=["voisin_insee"])  # garder seulement les lignes avec un voisin

# === 7) Joindre pour récupérer les infos de la banlieue (voisine) ===
# On va chercher, pour chaque voisin_insee, son nom et ses codes postaux
banlieue_infos = communes_simple.rename(columns={
    "code_insee": "code_insee_banlieue",
    "nom_standard": "banlieue",
    "codes_postaux": "codes_postaux_banlieue"
})

adj_avec_banlieue = adj_long.merge(
    banlieue_infos,
    left_on="voisin_insee",
    right_on="code_insee_banlieue",
    how="left"
)

# === 8) Joindre pour récupérer les infos de la commune source + son code postal (explosé) ===
# On relie le code INSEE source (adj["insee"]) au code INSEE des communes (communes_cp["code_insee"])
final = adj_avec_banlieue.merge(
    communes_cp[["code_insee", "commune", "code_postal"]],
    left_on="insee",
    right_on="code_insee",
    how="left"
)

# === 9) Ne garder que les colonnes finales, enlever doublons et lignes incomplètes ===
banlieues_df = final[[
    "code_postal",           # CP de la commune source
    "commune",               # Nom de la commune source
    "insee",                 # INSEE source
    "banlieue",              # Nom de la commune voisine (banlieue)
    "code_insee_banlieue",   # INSEE de la banlieue
    "codes_postaux_banlieue" # CP(s) de la banlieue (séparés par '|')
]].rename(columns={"insee": "code_insee_source"})

# On enlève les lignes où il manque l'essentiel
banlieues_df = banlieues_df.dropna(subset=["code_postal", "commune", "banlieue"])
banlieues_df = banlieues_df.drop_duplicates()

# === Filtrer une zone, ex. Île-de-France (75, 92, 93, 94) ===, Paris, Marseille, 
banlieues_df = banlieues_df[banlieues_df["code_postal"].str.match(r"^(75|13|59|49|80|44|38)")]

# ---  Normaliser les codes postaux  ---
banlieues_df["code_postal"] = (
    banlieues_df["code_postal"]
    .astype(str)
    .str.extract(r"(\d{2,5})", expand=False)  # récupère les chiffres principaux
    .fillna("")
    .str.zfill(5)
)

# ---  Filtrer : ne garder que les lignes dont le CP SOURCE est dans ta liste ---
cp_set = set(code_postal_ville)
extrait_source = banlieues_df[banlieues_df["code_postal"].isin(cp_set)].copy()

# ---  inclure aussi si la BANLIEUE possède un CP dans ta liste ---
def banlieue_a_cp_dans_liste(cell, cp_set):
    """Retourne True si au moins un CP (séparés par '|') de la banlieue est dans code_postal_ville."""
    if pd.isna(cell) or str(cell).strip() == "":
        return False
    return any(cp.strip().zfill(5) in cp_set for cp in str(cell).split("|"))

mask_banlieue = banlieues_df["codes_postaux_banlieue"].apply(lambda x: banlieue_a_cp_dans_liste(x, cp_set))
extrait_banlieue = banlieues_df[mask_banlieue].copy()
code_postal_ville.extend(extrait_banlieue["code_postal"].tolist())
dfliste = pd.Series(code_postal_ville)

outfile = "../DataCleaned/metropoleetbanlieues_liste.csv"
dfliste.to_csv(outfile, index=False, encoding="utf-8")


In [ ]:
chunks = pd.read_csv('../Data/Culture.csv', chunksize=1000000, low_memory=False, sep=";")


chunk_list = []
for chunk in chunks:
    print("Processing a new chunk...")
    df1 = pd.DataFrame(chunk)
    df2 = df1[['Code_Postal']].dropna()
    chunk_list.append(df2)

# Concatenate all chunks into a single DataFrame
df = pd.concat(chunk_list, ignore_index=True)

# Display basic information about the dataset
counts = df.groupby("Code_Postal").size().reset_index(name="CultureCount")
print(counts.sort_values(by="CultureCount", ascending=False).head(10))

counts.to_csv('../DataCleaned/Culture_Cleaned.csv', index=False)

In [ ]:
# Load the CSV
chunks = pd.read_csv('../Data/ValeurFonciere.csv', chunksize=1000000, low_memory=False, sep=",")

chunk_list = []
for chunk in chunks:
    print("Processing a new chunk...")
    df1 = pd.DataFrame(chunk)
    df1 = df1[['nature_mutation','valeur_fonciere','code_postal','surface_reelle_bati','nombre_pieces_principales','type_local','longitude','latitude','date_mutation']]
    df1 = df1.dropna(subset=['code_postal'])
    df1['code_postal'] = df1['code_postal'].astype(float).astype(int).astype(str).str.zfill(5)
    df1['date_mutation'] = pd.to_datetime(df1['date_mutation'])
    chunk_list.append(df1)

# Concatenate all chunks into a single DataFrame
dftot = pd.concat(chunk_list, ignore_index=True)

# Display basic information about the dataset
print(dftot.info())
print(dftot.head())

dftot.to_csv('../DataCleaned/DVF_Cleaned.csv', index=False)

In [ ]:
# Load the CSV
chunks = pd.read_csv('../Data/Education.csv', chunksize=1000000, low_memory=False, sep=";")

chunk_list = []
for chunk in chunks:
    print("Processing a new chunk...")
    df1 = pd.DataFrame(chunk)
    df2 = df1[['Adresse : code postal']].dropna().copy()
    df2 = df2.rename(columns={'Adresse : code postal':'CodePostal'})
    df2['CodePostal'] = df2['CodePostal'].apply(lambda value: str(value).zfill(5))
    chunk_list.append(df2)

# Concatenate all chunks into a single DataFrame
df = pd.concat(chunk_list, ignore_index=True)
count = df.groupby("CodePostal").size().reset_index(name="EducationCount")

# Display basic information about the dataset
print(count.info())
print(count.head())
print(count.sort_values(by="EducationCount", ascending=False).head(10))
count.to_csv('../DataCleaned/EducationCount_Cleaned.csv', index=False)

In [ ]:
# Load the CSV
chunks = pd.read_csv('../Data/LoyerT1T2.csv', chunksize=1000000, low_memory=False, sep=";", encoding="latin1")

chunk_list = []
for chunk in chunks:
    print("Processing a new chunk...")
    df1 = pd.DataFrame(chunk)
    df2 = df1[['DEP', 'loypredm2']].dropna().copy()
    df2 = df2.rename(columns={'DEP':'Département','loypredm2':'Loyerm2'})
    df2['Loyerm2'] = df2['Loyerm2'].astype(str).str.replace(',', '.').astype(float)
    chunk_list.append(df2)

# Concatenate all chunks into a single DataFrame
df = pd.concat(chunk_list, ignore_index=True)
df['Loyerm2'] = pd.to_numeric(df['Loyerm2'], errors='coerce')
mean = df.groupby("Département")["Loyerm2"].mean().reset_index(name="Loyerm2Mean")

# Display basic information about the dataset
print(mean.sort_values(by="Loyerm2Mean", ascending=False).head())
mean.to_csv('../DataCleaned/LoyerT1T2_Cleaned.csv', index=False)

In [ ]:
# stop_lat,stop_lon

# Load the CSV
df = pd.read_csv("../Data/latlongCP.csv", sep=",",low_memory=False)
chunks = pd.read_csv('../Data/Transport.csv', chunksize=1000000, low_memory=False, sep=",")

# Extract lat/lon as radians (for haversine distance)
df["lat"] = df["latitude"].astype(float)
df["lon"] = df["longitude"].astype(float)
df = df.dropna(subset=["lat", "lon", "code_postal"])
df = df.dropna()
coords_rad = np.radians(df[["lat", "lon"]].to_numpy())

# Build BallTree (fast nearest-neighbor search)
tree = BallTree(coords_rad, metric="haversine")

def latlon_to_zip(lat, lon):
    """Return closest postal code for given latitude, longitude."""
    dist, idx = tree.query(np.radians([[lat, lon]]), k=1)
    postal = df.iloc[idx[0][0]]["code_postal"]
    postal = str(postal).zfill(5)
    return str(postal)

chunk_list = []
for chunk in chunks:
    print("Processing a new chunk...")
    df1 = pd.DataFrame(chunk)
    df2 = df1[['stop_lat','stop_lon']].dropna()
    df3 = pd.DataFrame()
    df3['CodePostal'] = df2.apply(lambda row: latlon_to_zip(row['stop_lat'], row['stop_lon']), axis=1)
    chunk_list.append(df3)

# Concatenate all chunks into a single DataFrame
df = pd.concat(chunk_list, ignore_index=True)

# Display basic information about the dataset
counts = df3.groupby("CodePostal").size().reset_index(name="TransportCount")
print(counts.sort_values(by="TransportCount", ascending=False).head(10))

counts.to_csv('../DataCleaned/TransportCount_Cleaned.csv', index=False)

Les données sont nettoyées.
Un version de tous chaque dataset est dissponible via le lien Google Drive suivant : https://drive.google.com/file/d/1l78W2CqDkuUzNHIQrd7voMC0O00ExEa7/view?usp=sharing

### Data join

In [ ]:
df_all_temp = {}
for dataset in ['Culture_Cleaned','Carburant_Cleaned','EducationCount_Cleaned','TransportCount_Cleaned','LoyerT1T2_Cleaned','metropoleetbanlieues_liste','DVF_Cleaned']:
    df_all_temp[dataset] = pd.read_csv('../DataCleaned/'+dataset+'.csv', low_memory=False, sep=",")

df_all = df_all_temp['DVF_Cleaned'].merge(df_all_temp['Carburant_Cleaned'], left_on="code_postal", right_on="code_postal")
df_all_temp.pop('DVF_Cleaned')
df_all_temp.pop('Carburant_Cleaned')
df_all = df_all.merge(df_all_temp['metropoleetbanlieues_liste'], left_on="code_postal", right_on="0", how='left')
df_all_temp.pop('metropoleetbanlieues_liste')
df_all = df_all.merge(df_all_temp['EducationCount_Cleaned'], left_on="code_postal", right_on="CodePostal", how='left')
df_all_temp.pop('EducationCount_Cleaned')
df_all = df_all.merge(df_all_temp['TransportCount_Cleaned'], left_on="code_postal", right_on="CodePostal", how='left')
df_all_temp.pop('TransportCount_Cleaned')
df_all['Département'] = df_all['code_postal'].apply(lambda x : str(x).zfill(5)).str[:2]
df_all = df_all.merge(df_all_temp['LoyerT1T2_Cleaned'], left_on="Département", right_on="Département", how='left')
df_all_temp.pop('LoyerT1T2_Cleaned')
df_all['CodePostal'] = df_all['code_postal'].apply(lambda x : str(x).zfill(5))
df_all = df_all.merge(df_all_temp['Culture_Cleaned'], left_on="CodePostal", right_on="Code_Postal", how='left')
df_all_temp.pop('Culture_Cleaned')

print(df_all.head())

df_all.to_csv('../DataAll/Dataset.csv', index=False)

NameError: name 'pd' is not defined

Une fois les données rassemblée, nous pouvons les utiliser pour faire l'analyse.
Une version de ce dataset join est disponible via le lien Google Drive suivant : https://drive.google.com/file/d/1XiIpsO4sZ9qwRYCcc6EtoccFavcKQEwE/view?usp=sharing